In [44]:
#Importamos las librerías a utilizar
import pandas as pd
import numpy as np
from scipy import special

## Inciso 1:
A continuación se leen y cargan los datos más actuales de COVID-19 por país, usando la librería pandas. 

In [49]:
#Importamos los datos que vamos a utilizar
datos = pd.read_csv('datos_covid/country_wise_latest.csv')

## Inciso 2
Se imprime la información del dataframe para determinar si es necesario hacer cambio de nombres a las columnas o al tipo de datos. Además, se imprime una fila de datos para analizarlos y escoger los mejores nombres para dichas columnas.

In [46]:
print(datos.info())
datos.loc[datos['Country/Region'] == 'Guatemala']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country/Region          187 non-null    object 
 1   Confirmed               187 non-null    int64  
 2   Deaths                  187 non-null    int64  
 3   Recovered               187 non-null    int64  
 4   Active                  187 non-null    int64  
 5   New cases               187 non-null    int64  
 6   New deaths              187 non-null    int64  
 7   New recovered           187 non-null    int64  
 8   Deaths / 100 Cases      187 non-null    float64
 9   Recovered / 100 Cases   187 non-null    float64
 10  Deaths / 100 Recovered  187 non-null    float64
 11  Confirmed last week     187 non-null    int64  
 12  1 week change           187 non-null    int64  
 13  1 week % increase       187 non-null    float64
 14  WHO Region              187 non-null    ob

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
70,Guatemala,45309,1761,32455,11093,256,27,843,3.89,71.63,5.43,39039,6270,16.06,Americas


Tras el análisis, se pproporciona una descripción probable de los datos en cada una de las columnas.
- Country/Region: País sobre el cual se presenta la información.
- Confirmed: Número de casos confirmados a la fecha
- Recovered: Número de casos recuperados a la fecha
- Active: Número de casos positivos vivos.
- New cases: Cantidad de casos confirmados del último día.
- New deaths: Cantidad de muertes confirmadas del último día.
- New recovered: Cantidad de casos recuperados del último día.

Con base en esto, se nombran las columnas de la siguiente forma.

In [50]:
datos.columns = ['País', 'Confirmados', 'Muertes', 'Recuperados', 'Activos', 'Nuevos casos', 'Nuevas muertes', 'Nuevos recuperados',\
                 'Tasa de mortalidad (%)', 'Tasa de recuperación (%)', 'Muertes/Recuperados (%)',\
                      'Confirmados a la semana pasada', 'Confirmados de la semana', 'Crecimiento de la última semana (%)','Región OMS']

In [51]:
#Seleccionamos los datos que tienen alguna columna vacía para determinar el procesamiento adecuado.

datos.loc[datos.isnull().any(axis = 1)]

,País,Confirmados,Muertes,Recuperados,Activos,Nuevos casos,Nuevas muertes,Nuevos recuperados,Tasa de mortalidad (%),Tasa de recuperación (%),Muertes/Recuperados (%),Confirmados a la semana pasada,Confirmados de la semana,Crecimiento de la última semana (%),Región OMS


In [52]:
#Verificamos que hay una única entrada para cada país.
datos.loc[datos.duplicated('País')]

,País,Confirmados,Muertes,Recuperados,Activos,Nuevos casos,Nuevas muertes,Nuevos recuperados,Tasa de mortalidad (%),Tasa de recuperación (%),Muertes/Recuperados (%),Confirmados a la semana pasada,Confirmados de la semana,Crecimiento de la última semana (%),Región OMS


In [53]:
#Se escoge el index de los datos como el país, pues este campo es único.
datos = datos.set_index('País')


## Inciso 3
Se calculan medidas importantes para los datos

## Inciso 4
Se crea una nueva columna llamada 'Active' que es la cantidad de pacientes (casos positivos no recuperados) vivos.

(Esta columna debería ser idéntica a la columna de 'Activos')

In [70]:
datos['Active'] = datos['Confirmados'] - datos['Recuperados'] - datos['Muertes']
datos[['Activos','Active']]

,Activos,Active
País,,
Afghanistan,9796,9796
Albania,1991,1991
Algeria,7973,7973
Andorra,52,52
Angola,667,667
...,...,...
West Bank and Gaza,6791,6791
Western Sahara,1,1
Yemen,375,375


## Inciso 5
Se agrupan por país y se muestra el total  de confirmados, muertes, recuperados y activos.
(Puesto que el país es único, el dataframe procesado debería ser el mismo, por lo que se incluye el mismo ejercicio pero agrupando por región de la OMS)

In [67]:
agrupados_pais = datos.groupby(datos.index).agg(Confirmados = ('Confirmados','sum'),\
                                            Muertes = ('Muertes', 'sum'),\
                                            Recuperados = ('Recuperados', 'sum'),\
                                            Activos = ('Activos', 'sum'))
agrupados_pais

,Confirmados,Muertes,Recuperados,Activos
País,,,,
Afghanistan,36263,1269,25198,9796
Albania,4880,144,2745,1991
Algeria,27973,1163,18837,7973
Andorra,907,52,803,52
Angola,950,41,242,667
...,...,...,...,...
West Bank and Gaza,10621,78,3752,6791
Western Sahara,10,1,8,1
Yemen,1691,483,833,375


In [68]:
#Repitiendo el ejercicio para las regiones de la OMS
agrupados_reg = datos.groupby('Región OMS').agg(Confirmados = ('Confirmados','sum'),\
                                            Muertes = ('Muertes', 'sum'),\
                                            Recuperados = ('Recuperados', 'sum'),\
                                            Activos = ('Activos', 'sum'))
agrupados_reg

,Confirmados,Muertes,Recuperados,Activos
Región OMS,,,,
Africa,723207,12223,440645,270339
Americas,8839286,342732,4468616,4027938
Eastern Mediterranean,1490744,38339,1201400,251005
Europe,3299523,211144,1993723,1094656
South-East Asia,1835297,41349,1156933,637015
Western Pacific,292428,8249,206770,77409


## Inciso 6
Se ordenan los países por el número de casos y se muestran los 10 con mayor cantidad.

In [73]:
#Se ordenan los datos por número de confirmados y se muestran los mayores 10.
datos_ordenados = datos.sort_values('Confirmados', ascending= False)
datos_ordenados.head(10)

,Confirmados,Muertes,Recuperados,Activos,Nuevos casos,Nuevas muertes,Nuevos recuperados,Tasa de mortalidad (%),Tasa de recuperación (%),Muertes/Recuperados (%),Confirmados a la semana pasada,Confirmados de la semana,Crecimiento de la última semana (%),Región OMS,Active
País,,,,,,,,,,,,,,,
US,4290259,148011,1325804,2816444,56336,1076,27941,3.45,30.90,11.16,3834677,455582,11.88,Americas,2816444
Brazil,2442375,87618,1846641,508116,23284,614,33728,3.59,75.61,4.74,2118646,323729,15.28,Americas,508116
India,1480073,33408,951166,495499,44457,637,33598,2.26,64.26,3.51,1155338,324735,28.11,South-East Asia,495499
Russia,816680,13334,602249,201097,5607,85,3077,1.63,73.74,2.21,776212,40468,5.21,Europe,201097
South Africa,452529,7067,274925,170537,7096,298,9848,1.56,60.75,2.57,373628,78901,21.12,Africa,170537
Mexico,395489,44022,303810,47657,4973,342,8588,11.13,76.82,14.49,349396,46093,13.19,Americas,47657
Peru,389717,18418,272547,98752,13756,575,4697,4.73,69.93,6.76,357681,32036,8.96,Americas,98752
Chile,347923,9187,319954,18782,2133,75,1859,2.64,91.96,2.87,333029,14894,4.47,Americas,18782
United Kingdom,301708,45844,1437,254427,688,7,3,15.19,0.48,3190.26,296944,4764,1.60,Europe,254427


## Inciso 7
Se muestra una gráfica de líneas con los casos, muertes y recuperaciones de un país específico.

In [76]:
datos2 = pd.read_csv('datos_covid/covid_19_clean_complete.csv')
datos2.loc[datos2['Country/Region'] == 'Guatemala']

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
121,NaN,Guatemala,15.7835,-90.2308,2020-01-22,0,0,0,0,Americas
382,NaN,Guatemala,15.7835,-90.2308,2020-01-23,0,0,0,0,Americas
643,NaN,Guatemala,15.7835,-90.2308,2020-01-24,0,0,0,0,Americas
904,NaN,Guatemala,15.7835,-90.2308,2020-01-25,0,0,0,0,Americas
1165,NaN,Guatemala,15.7835,-90.2308,2020-01-26,0,0,0,0,Americas
...,...,...,...,...,...,...,...,...,...,...
47884,NaN,Guatemala,15.7835,-90.2308,2020-07-23,42192,1632,28856,11704,Americas
48145,NaN,Guatemala,15.7835,-90.2308,2020-07-24,43283,1669,30150,11464,Americas
48406,NaN,Guatemala,15.7835,-90.2308,2020-07-25,44492,1699,31045,11748,Americas
48667,NaN,Guatemala,15.7835,-90.2308,2020-07-26,45053,1734,31612,11707,Americas
